In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt

from datetime import datetime
from google.colab import drive
from pandas_datareader import data as pdr
from sklearn.metrics import mean_squared_error
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Mounted at /content/drive


# Run Logit Regression on Training set



In [ ]:
Training = pd.read_csv('/content/drive/MyDrive/Senior Fall/Senior Thesis/Data/Separately/Train.csv', converters={'Global Company Key':str})

In [ ]:
Train_y = Training['Selected']
Train_X = Training.drop(columns=['Selected'])

model = LogisticRegression()
model.fit(Train_X, Train_y)

LogisticRegression()

# Predict Logit Regression using Test set

In [ ]:
Test = pd.read_csv('/content/drive/MyDrive/Senior Fall/Senior Thesis/Data/Separately/Test_filled_and_cleaned.csv', index_col=0, converters={'Global Company Key':str})

In [ ]:
predictions = model.predict_proba(Test[Train_X.columns])
Test['Selected']= [row[1] for row in predictions]

# Sort them into quintiles by 'Data Date' and then by 'Selected'

In [ ]:
Test = Test.sort_values(by=['Data Date', 'Selected'], ascending = [True, False])

In [ ]:
def obtain_Ticker(group):
    return group.head(len(group) // 5)['Ticker Symbol'].tolist()

In [ ]:
# Group DataFrame by 'Data Date' and apply the custom function
Tickers = Test.groupby('Data Date').apply(obtain_Ticker)
Dates = Test['Data Date'].unique()

In [ ]:
replicating_portfolio1 = pd.DataFrame()

for i in range(300):
  print(i)
#for i in range(36):
  start_date = Dates[i]

  if (i+1) < len(Dates):
    end_date = (datetime.strptime(Dates[i+1], '%Y-%m-%d') + relativedelta(days=1)).strftime('%Y-%m-%d')
  else:
    end_date = (datetime.strptime(Dates[i], '%Y-%m-%d') + relativedelta(months=1) + relativedelta(days=1)).strftime('%Y-%m-%d')

  df = pd.DataFrame()
  print(start_date)
  print(end_date)

  for ticker in Tickers[i]:
    try:
      ticker_data = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
      if not ticker_data.empty:
        df[ticker] = ticker_data
        df[ticker] = df[ticker].fillna(method='ffill')
        df[ticker] = df[ticker].pct_change()
        df['Equal_Weighted_Returns'] = df.mean(axis=1)
    except Exception as e:
      print('exception e')

  if not df.empty:
    replicating_portfolio1 = pd.concat([replicating_portfolio1, df['Equal_Weighted_Returns']], axis=0)

replicating_portfolio1

In [ ]:
replicating_portfolio1.to_csv('Logit_rep1.csv')

In [ ]:
replicating_portfolio2 = pd.DataFrame()

for i in range(300, 400):
  print(i)
#for i in range(36):
  start_date = Dates[i]

  if (i+1) < len(Dates):
    end_date = (datetime.strptime(Dates[i+1], '%Y-%m-%d') + relativedelta(days=1)).strftime('%Y-%m-%d')
  else:
    end_date = (datetime.strptime(Dates[i], '%Y-%m-%d') + relativedelta(months=1) + relativedelta(days=1)).strftime('%Y-%m-%d')

  df = pd.DataFrame()
  print(start_date)
  print(end_date)

  for ticker in Tickers[i]:
    try:
      ticker_data = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
      if not ticker_data.empty:
        df[ticker] = ticker_data
        df[ticker] = df[ticker].fillna(method='ffill')
        df[ticker] = df[ticker].pct_change()
        df['Equal_Weighted_Returns'] = df.mean(axis=1)
    except Exception as e:
      print('exception e')

  if not df.empty:
    replicating_portfolio2 = pd.concat([replicating_portfolio2, df['Equal_Weighted_Returns']], axis=0)

replicating_portfolio2

In [ ]:
replicating_portfolio2.to_csv('Logit_rep2.csv', index=True)

In [ ]:
replicating_portfolio3 = pd.DataFrame()

for i in range(400, len(Tickers)):
  print(i)
#for i in range(36):
  start_date = Dates[i]

  if (i+1) < len(Dates):
    end_date = (datetime.strptime(Dates[i+1], '%Y-%m-%d') + relativedelta(days=1)).strftime('%Y-%m-%d')
  else:
    end_date = (datetime.strptime(Dates[i], '%Y-%m-%d') + relativedelta(months=1) + relativedelta(days=1)).strftime('%Y-%m-%d')

  df = pd.DataFrame()
  print(start_date)
  print(end_date)

  for ticker in Tickers[i]:
    try:
      ticker_data = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
      if not ticker_data.empty:
        df[ticker] = ticker_data
        df[ticker] = df[ticker].fillna(method='ffill')
        df[ticker] = df[ticker].pct_change()
        df['Equal_Weighted_Returns'] = df.mean(axis=1)
    except Exception as e:
      print('exception e')

  if not df.empty:
    replicating_portfolio3 = pd.concat([replicating_portfolio3, df['Equal_Weighted_Returns']], axis=0)

df
replicating_portfolio3

In [ ]:
replicating_portfolio3.to_csv('Logit_rep3.csv', index=True)

# We need to concat the three csvs together

In [ ]:
rep1 = pd.read_csv('/content/drive/MyDrive/Senior Fall/Senior Thesis/Data/Separately/Logit_rep1.csv')
rep2 = pd.read_csv('/content/drive/MyDrive/Senior Fall/Senior Thesis/Data/Separately/Logit_rep2.csv')
rep3 = pd.read_csv('/content/drive/MyDrive/Senior Fall/Senior Thesis/Data/Separately/Logit_rep3.csv')

In [ ]:
rep = pd.concat([rep1, rep2, rep3])
rep = rep.rename(columns={'0': 'returns'})
rep = rep.dropna()

In [ ]:
rep.to_csv('Logit_replicating.csv')

# Below we run a Sparse Logit Regression

In [ ]:
X = pd.read_csv('/content/drive/MyDrive/Senior Fall/Senior Thesis/Data/Separately/Train_big.csv', converters={'Global Company Key':str})

In [ ]:
X.drop(X.columns[-1], axis=1, inplace=True)

In [ ]:
X_train, X_test = train_test_split(X, test_size=0.25, random_state=42)

y_train = X_train['Selected']
X_train = X_train.drop(columns=['Selected'])

y_test = X_test['Selected']
X_test = X_test.drop(columns=['Selected'])

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
lambda_values = [0.01, 0.1, 1.0, 10.0]
results = {
    'lambda': [],
    'accuracy': [],
    'MAE': [],
    'RMSE': []
}
for lambda_val in lambda_values:
  print(lambda_val)
  # Creating a logistic regression model with L1 penalty (for sparsity) and setting C = 1/lambda
  model = LogisticRegression(penalty='l1', C=1/lambda_val, solver='liblinear', random_state=42)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)

  # Calculating accuracy
  accuracy = accuracy_score(y_test, y_pred)

  # Calculating mean absolute error
  mae = mean_absolute_error(y_test, y_pred)

  # Calculating root mean squared error
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))

  # Storing results
  results['lambda'].append(lambda_val)
  results['accuracy'].append(accuracy)
  results['MAE'].append(mae)
  results['RMSE'].append(rmse)

# Printing results
print("Results:")
print("Lambda   Accuracy   MAE       RMSE")

for i in range(len(lambda_values)):
    print(f"{results['lambda'][i]:.2f}     {results['accuracy'][i]:.4f}     {results['MAE'][i]:.4f}     {results['RMSE'][i]:.4f}")

0.01


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.1


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1.0
10.0
Results:
Lambda   Accuracy   MAE       RMSE
0.01     0.9970     0.0030     0.0552
0.10     0.9970     0.0030     0.0543
1.00     0.9969     0.0031     0.0555
10.00     0.9970     0.0030     0.0543
